In [2]:
import warnings
import pandas as pd
import pyodbc
import numpy as np

warnings.filterwarnings('ignore')  # Clean up output

# --- CONFIGURATION ---
# Note: If running inside Docker, use 'sql_server'.
# If running on Windows, use 'localhost'.
CONN_STR = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost,1433;"
    "UID=sa;"
    "PWD=MyComplexPassword123!;"
    "TrustServerCertificate=yes;"
)


def get_data():
    """Fetch and preprocess data from SQL Server."""
    print("🔌 Connecting to SQL Server...")
    try:
        conn = pyodbc.connect(CONN_STR)

        # Order by date ASC to ensure charts draw correctly
        query = """
        SELECT full_date, Price, news_count, avg_conflict_score 
        FROM Gold_Analytics_RedSea 
        ORDER BY full_date ASC
        """
        df = pd.read_sql(query, conn)
        conn.close()

        # --- PRE-PROCESSING ---
        # 1. Convert to Datetime (Crucial for Time Series)
        df['full_date'] = pd.to_datetime(df['full_date'])

        # 2. Calculate Risk Score
        # Logic: News Volume * Intensity.
        # Since avg_conflict_score is already positive (High=Bad), we just multiply.
        df['Risk_Score'] = df['news_count'] * df['avg_conflict_score']

        # 3. Rounding for cleaner displays
        df['Price'] = df['Price'].round(2)
        df['Risk_Score'] = df['Risk_Score'].round(2)

        return df

    except Exception as e:
        print(f"❌ Error: {e}")
        return pd.DataFrame()


if __name__ == "__main__":
    # Load Data into DataFrame
    df = get_data()

    print(f"✅ SUCCESS: Loaded {len(df)} rows.")
    print(f"📅 Date Range: {df['full_date'].min()} to {df['full_date'].max()}")
    print("--- Sample Data ---")
    print(df.head())

🔌 Connecting to SQL Server...
❌ Error: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: The wait operation timed out.\r\n (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (258)')
✅ SUCCESS: Loaded 0 rows.


KeyError: 'full_date'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data
df = get_data()

# --- VISUAL CONFIGURATION ---
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 120  # High resolution for reports
plt.rcParams['font.family'] = 'sans-serif'

# Create a figure with subplots (2 Rows, 2 Columns)
fig = plt.figure(figsize=(15, 10))
fig.suptitle(
    'Exploratory Data Analysis: Geopolitical Risk vs. Logistics Cost',
    fontsize=16,
    y=0.95
)

# --- PLOT 1: DISTRIBUTION OF RISK (HISTOGRAM) ---
ax1 = fig.add_subplot(2, 2, 1)
sns.histplot(
    df['Risk_Score'],
    bins=40,
    kde=True,
    color='#3498db',
    ax=ax1
)
ax1.set_title(
    'Distribution of Daily Risk Scores (The "Long Tail" of Crisis)',
    fontsize=12
)
ax1.set_xlabel('Risk Magnitude')
ax1.set_ylabel('Frequency')

# --- PLOT 2: CORRELATION HEATMAP ---
ax2 = fig.add_subplot(2, 2, 2)
# Select only numeric columns
corr_data = df[[
    'Price',
    'news_count',
    'avg_conflict_score',
    'Risk_Score'
]].corr()

sns.heatmap(
    corr_data,
    annot=True,
    cmap='RdBu_r',
    fmt=".2f",
    linewidths=1,
    vmin=-1,
    vmax=1,
    ax=ax2
)
ax2.set_title('Pearson Correlation Matrix', fontsize=12)

# --- PLOT 3: REGRESSION ANALYSIS ---
ax3 = fig.add_subplot(2, 2, 3)
sns.regplot(
    data=df,
    x='Risk_Score',
    y='Price',
    scatter_kws={'alpha': 0.3, 'color': '#2c3e50', 's': 10},
    line_kws={'color': '#e74c3c'},
    ax=ax3
)
ax3.set_title('Regression: Impact of Risk Intensity on Price', fontsize=12)
ax3.set_xlabel('Calculated Risk Intensity')
ax3.set_ylabel('Shipping Price ($)')

# --- PLOT 4: PRICE VOLATILITY (BOX PLOT) ---
ax4 = fig.add_subplot(2, 2, 4)
# Create a "Crisis" vs "Peace" categorical column for comparison
df['Period'] = df['full_date'].apply(
    lambda x: 'Crisis (2024)' if x.year >= 2024 else 'Baseline (2023)'
)
sns.boxplot(data=df, x='Period', y='Price', palette="Set2", ax=ax4)
ax4.set_title('Price Volatility: Baseline vs. Crisis', fontsize=12)

plt.tight_layout(pad=3.0)
plt.show()

# --- STATISTICAL SUMMARY FOR TEXT ---
print("--- STATISTICAL FINDINGS ---")
correlation = df['Risk_Score'].corr(df['Price'])
print(f"1. Correlation Coefficient (r): {correlation:.3f}")
print(
    "   (Interpretation: Strong positive relationship between news risk and price)"
)
print(f"2. Max Risk Score: {df['Risk_Score'].max()}")
print(f"3. Max Price: ${df['Price'].max()}")

In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Load Data
df = get_data()

# --- DASHBOARD APP ---
app = Dash(
    __name__,
    external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
)

# Styles
card_style = {
    'backgroundColor': 'white',
    'borderRadius': '10px',
    'padding': '20px',
    'boxShadow': '0 4px 6px rgba(0,0,0,0.1)',
    'marginBottom': '20px'
}

app.layout = html.Div(
    style={
        'backgroundColor': '#f4f6f8',
        'padding': '20px',
        'fontFamily': 'Segoe UI'
    },
    children=[
        # Header
        html.Div(
            children=[
                html.H2(
                    "🛑 Red Sea Risk Radar",
                    style={'color': 'white', 'marginBottom': '5px'}
                ),
                html.P(
                    "Executive Control Tower: Supply Chain Disruption Signals",
                    style={'color': '#bdc3c7'}
                )
            ],
            style={
                'backgroundColor': '#2c3e50',
                'padding': '20px',
                'borderRadius': '10px',
                'marginBottom': '20px'
            }
        ),

        # Controls
        html.Div(
            children=[
                html.Label(
                    "📅 Select Strategic Window:",
                    style={'fontWeight': 'bold'}
                ),
                dcc.DatePickerRange(
                    id='date-picker',
                    min_date_allowed=df['full_date'].min(),
                    max_date_allowed=df['full_date'].max(),
                    start_date=pd.to_datetime("2023-11-15"),
                    end_date=pd.to_datetime("2024-01-31"),
                    display_format='YYYY-MM-DD'
                )
            ],
            style={'textAlign': 'center', 'marginBottom': '20px'}
        ),

        # KPI Row
        html.Div(
            className='row',
            children=[
                html.Div(
                    children=[
                        html.H6("PEAK PRICE"),
                        html.H2(id='kpi-price')
                    ],
                    className='four columns',
                    style=card_style
                ),
                html.Div(
                    children=[
                        html.H6("TOTAL EVENTS"),
                        html.H2(id='kpi-news')
                    ],
                    className='four columns',
                    style=card_style
                ),
                html.Div(
                    children=[
                        html.H6("CORRELATION"),
                        html.H2(id='kpi-corr')
                    ],
                    className='four columns',
                    style=card_style
                ),
            ]
        ),

        # Main Chart
        html.Div(
            children=[
                dcc.Graph(id='trend-chart', style={'height': '500px'})
            ],
            style=card_style
        ),

        # Bottom Charts
        html.Div(
            className='row',
            children=[
                html.Div(
                    children=[
                        html.H5(
                            "Panic Meter (Clusters)",
                            style={'textAlign': 'center'}
                        ),
                        dcc.Graph(id='scatter-chart')
                    ],
                    className='six columns',
                    style=card_style
                ),
                html.Div(
                    children=[
                        html.H5(
                            "Risk Signal Volatility",
                            style={'textAlign': 'center'}
                        ),
                        dcc.Graph(id='risk-line-chart')
                    ],
                    className='six columns',
                    style=card_style
                ),
            ]
        )
    ]
)


# Callbacks
@app.callback(
    [
        Output('trend-chart', 'figure'),
        Output('scatter-chart', 'figure'),
        Output('risk-line-chart', 'figure'),
        Output('kpi-price', 'children'),
        Output('kpi-news', 'children'),
        Output('kpi-corr', 'children')
    ],
    [
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date')
    ]
)
def update_dashboard(start, end):
    mask = (df['full_date'] >= start) & (df['full_date'] <= end)
    filtered = df.loc[mask]

    if filtered.empty:
        return (
            go.Figure(),
            go.Figure(),
            go.Figure(),
            "-",
            "-",
            "-"
        )

    # 1. Trend
    fig1 = make_subplots(specs=[[{"secondary_y": True}]])
    fig1.add_trace(
        go.Bar(
            x=filtered['full_date'],
            y=filtered['Risk_Score'],
            name="Risk Intensity",
            marker_color='rgba(231, 76, 60, 0.6)'
        ),
        secondary_y=False
    )
    fig1.add_trace(
        go.Scatter(
            x=filtered['full_date'],
            y=filtered['Price'],
            name="Price",
            line=dict(color='#2c3e50', width=3)
        ),
        secondary_y=True
    )
    fig1.update_layout(
        title="Lag Analysis",
        template="plotly_white",
        legend=dict(orientation="h", y=1.1)
    )
    fig1.update_yaxes(title="Risk Score", secondary_y=False, showgrid=False)
    fig1.update_yaxes(title="Price ($)", secondary_y=True, showgrid=True)

    # 2. Scatter
    fig2 = px.scatter(
        filtered,
        x='full_date',
        y='avg_conflict_score',
        size='news_count',
        color='Price',
        size_max=25,
        color_continuous_scale='RdBu_r'
    )
    fig2.update_layout(template="plotly_white")

    # 3. Risk Line
    fig3 = px.line(filtered, x='full_date', y='Risk_Score')
    fig3.update_traces(
        line_color='#e74c3c',
        line_width=2,
        fill='tozeroy',
        fillcolor='rgba(231, 76, 60, 0.1)'
    )
    fig3.update_layout(template="plotly_white")

    # KPIs
    kp = f"${filtered['Price'].max():,.0f}"
    kn = f"{filtered['news_count'].sum():,.0f}"

    if len(filtered) > 1:
        corr_val = filtered['Price'].corr(filtered['Risk_Score'])
        kc = f"{corr_val:.2f}"
    else:
        kc = "-"

    return fig1, fig2, fig3, kp, kn, kc


if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='inline', port=8055)